In [1]:
import pygame
import random

# --- Game Setup ---
pygame.init()
WIDTH, HEIGHT = 600, 600
FPS = 30
WHITE = (255, 255, 255)
GREY = (50, 50, 50)
BLUE = (65, 105, 225)
GREEN = (50, 205, 50)
FONT = pygame.font.SysFont('arial', 36)
SMALL_FONT = pygame.font.SysFont('arial', 24)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Memory Matching Game")
clock = pygame.time.Clock()

# --- Game State ---
level = 1
score = 0
max_level = 5

def create_board(level):
    size = 2 + level
    if size % 2 != 0:
        size += 1
    numbers = list(range(1, (size * size) // 2 + 1)) * 2
    random.shuffle(numbers)
    cards = [numbers[i * size:(i + 1) * size] for i in range(size)]
    revealed = [[False] * size for _ in range(size)]
    return cards, revealed, size

# Initialize game values
cards, revealed, size = create_board(level)
CARD_WIDTH = WIDTH // size
CARD_HEIGHT = HEIGHT // size
first_pick = None
second_pick = None
waiting = False
wait_start = 0
matches = 0
game_active = False
game_over = False
time_up = False

# Timer
timer = 30 + level * 5
start_ticks = pygame.time.get_ticks()

# --- Functions ---
def draw_grid():
    for row in range(size):
        for col in range(size):
            x = col * CARD_WIDTH
            y = row * CARD_HEIGHT
            rect = pygame.Rect(x, y, CARD_WIDTH, CARD_HEIGHT)

            if revealed[row][col]:
                pygame.draw.rect(screen, GREEN, rect)
                num_surface = FONT.render(str(cards[row][col]), True, WHITE)
                screen.blit(num_surface, (x + CARD_WIDTH // 2 - num_surface.get_width() // 2,
                                          y + CARD_HEIGHT // 2 - num_surface.get_height() // 2))
            else:
                pygame.draw.rect(screen, BLUE, rect)
            pygame.draw.rect(screen, WHITE, rect, 3)

def get_card_pos(pos):
    x, y = pos
    return y // CARD_HEIGHT, x // CARD_WIDTH

def show_text(text, size=48, y_offset=0):
    font = pygame.font.SysFont('arial', size)
    text_surf = font.render(text, True, WHITE)
    screen.blit(text_surf, (WIDTH // 2 - text_surf.get_width() // 2, HEIGHT // 2 - text_surf.get_height() // 2 + y_offset))

# --- Main Game Loop ---
running = True

while running:
    screen.fill(GREY)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == pygame.MOUSEBUTTONDOWN:
            if not game_active:
                # Restart the game
                game_active = True
                time_up = False
                game_over = False
                level = 1
                score = 0
                timer = 30 + level * 5
                start_ticks = pygame.time.get_ticks()
                cards, revealed, size = create_board(level)
                CARD_WIDTH = WIDTH // size
                CARD_HEIGHT = HEIGHT // size
                first_pick = None
                second_pick = None
                waiting = False
                matches = 0

            elif game_active and not waiting and not game_over:
                row, col = get_card_pos(pygame.mouse.get_pos())
                if row < size and col < size and not revealed[row][col]:
                    if not first_pick:
                        first_pick = (row, col)
                        revealed[row][col] = True
                    elif not second_pick:
                        second_pick = (row, col)
                        revealed[row][col] = True
                        if cards[first_pick[0]][first_pick[1]] != cards[second_pick[0]][second_pick[1]]:
                            waiting = True
                            wait_start = pygame.time.get_ticks()
                        else:
                            score += 10
                            matches += 1
                            first_pick = None
                            second_pick = None

    # Handle mismatch delay
    if waiting and pygame.time.get_ticks() - wait_start > 1000:
        revealed[first_pick[0]][first_pick[1]] = False
        revealed[second_pick[0]][second_pick[1]] = False
        first_pick = None
        second_pick = None
        waiting = False

    if game_active and not game_over:
        # Timer logic
        elapsed = (pygame.time.get_ticks() - start_ticks) // 1000
        remaining_time = max(0, timer - elapsed)
        if remaining_time == 0:
            time_up = True
            game_active = False

        # Draw game
        draw_grid()
        score_text = SMALL_FONT.render(f"Level: {level}  Score: {score}", True, WHITE)
        time_text = SMALL_FONT.render(f"Time: {remaining_time}s", True, WHITE)
        screen.blit(score_text, (10, 10))
        screen.blit(time_text, (WIDTH - 160, 10))

        # Level complete
        if matches == (size * size) // 2:
            level += 1
            if level > max_level:
                game_over = True
            else:
                cards, revealed, size = create_board(level)
                CARD_WIDTH = WIDTH // size
                CARD_HEIGHT = HEIGHT // size
                matches = 0
                first_pick = None
                second_pick = None
                waiting = False
                timer = 30 + level * 5
                start_ticks = pygame.time.get_ticks()

    elif not game_active:
        if time_up:
            show_text("⏰ Time's Up!", 50, -40)
            show_text(f"Final Score: {score}", 36, 20)
            show_text("Click to Restart", 28, 70)
        elif game_over:
            show_text("🎉 You Win! 🎉", 50, -40)
            show_text(f"Final Score: {score}", 36, 20)
            show_text("Click to Restart", 28, 70)
        else:
            show_text("Memory Match Game", 50, -50)
            show_text("Click to Start", 30, 30)

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
